# **Import Statements**

In [1]:
import sys
root = "C:\\Users\\saman\\OneDrive\\Documents\\GitHub\\Eyelid-Modeling\\" #laptop path
#root = "C:\\Users\\Samantha\\Documents\\GitHub\\Eyelid-Modeling\\" #desktop path
sys.path.append(root)

from utils.experiment_manager import Experiment
from models.EMG_RNN_CNN import EMG_RNN_CNN_Wrapper
from utils.data_management import Mat2TVT

import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle

%load_ext autoreload
%autoreload 2

torch.cuda.is_available()

True

# **Data Reorganization** <br>


In [2]:
eye_bool = np.asarray([True, True, False, True, True, False, False, False])
electrode_list = ['u1', 'u2', 'u3', 'u4', 't2']
marker_list = ['u1', 'u2', 'u3', 'u4', 'u5']
filepath = root + 'eyeliddata.mat'

train = 0.7
val = 0.2
test = 0.1

reorganizer = Mat2TVT(eye_bool, electrode_list, marker_list, filepath)
_, _, identifier = reorganizer.load_data()
X, y = reorganizer.DMVC_norm()
TVT_dict = reorganizer.TVT_split(train, val, test)

In [31]:
rnn = torch.nn.RNN(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)

print(output.shape)

torch.Size([5, 3, 20])


# **Run Experiments**

In [ ]:
params = {
    't_win': np.arange(10,51,1, dtype = float),
    't_stride': np.arange(10, 51, 1, dtype = float),
    't_lookahead': np.arange(50, 101, 1, dtype = float),
    'win_mode': ['win_fft', 'win_rect'],
    'p_transform': np.linspace(0, 1, 100, dtype = float),
    'sigma': np.linspace(0, 5, 100, dtype = float),
    'p_mask': np.linspace(0, 0.5, 100, dtype = float),
    'batch_size': [4, 8, 16, 32, 64, 128],
    'learning_rate': np.linspace(1e-3, 1e-8, 500, dtype = float),
    'dropout': np.linspace(0, 1, 100, dtype = float),
    'RNN_hdim': [4, 8, 16, 32, 64, 128],
    'RNN_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
    'CNN_kernel': [3, 5]
}

experiment = Experiment()
model = EMG_RNN_CNN_Wrapper()
gcollect = True
best_params = experiment(params, TVT_dict, model, n_trials = 100, epochs = 100, patience = 25, gcollect) 

print("Best Params:\n_________________________________________________________________________________________________________")
for key, value in best_params.items():
    print('    {}: {}'.format(key, value))

[I 2025-03-14 04:03:14,857] A new study created in memory with name: no-name-21b9569d-7db6-4995-a7ee-aa3ac7679a4a


Epoch 1
-------------------------------


[I 2025-03-14 04:03:41,938] Trial 0 finished with value: 118.41309356689453 and parameters: {'t_win': np.float64(41.0), 't_stride': np.float64(11.0), 't_lookahead': np.float64(100.0), 'win_mode': 'win_rect', 'p_transform': np.float64(0.7373737373737375), 'sigma': np.float64(2.4242424242424243), 'p_mask': np.float64(0.13131313131313133), 'batch_size': 8, 'learning_rate': np.float64(0.00016233302605210412), 'dropout': np.float64(0.29292929292929293), 'RNN_hdim': 4, 'RNN_depth': 10, 'CNN_kernel': 5}. Best is trial 0 with value: 118.41309356689453.


Validation Error: 
 Max Marker Error: 3650.2%, Avg Marker Error: 118.4%, Avg loss: 1.071774 

Done!
Best Params:
_________________________________________________________________________________________________________
    t_win: 41.0
    t_stride: 11.0
    t_lookahead: 100.0
    win_mode: win_rect
    p_transform: 0.7373737373737375
    sigma: 2.4242424242424243
    p_mask: 0.13131313131313133
    batch_size: 8
    learning_rate: 0.00016233302605210412
    dropout: 0.29292929292929293
    RNN_hdim: 4
    RNN_depth: 10
    CNN_kernel: 5


# **Training Model with Best Params**

In [ ]:
metrics = experiment.run_experiment(best_params, TVT_dict, model, epochs = 1000, patience = 200)

'''
Saving Model, Training Metrics, and Best Params
'''
with open("training_metrics.pickle", "wb") as file:
    pickle.dump(metrics, file)
with open("best_params.pickle", "wb") as file:
    pickle.dump(best_params, file)

# **Plotting Loss Metrics**

In [ ]:
plt.plot(metrics['Training Loss'], label = 'Training Loss', marker = '.')
plt.plot(metrics['Validation Loss'], label = 'Validation Loss', marker = '.')
plt.yscale('log')
plt.title('Loss vs. Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(metrics['Training Avg Marker Error'], label = 'Training Error', marker = '.')
plt.plot(metrics['Validation Avg Marker Error'], label = 'Validation Error', marker = '.')
plt.yscale('log')
plt.title('Average Marker Error vs. Epoch')
plt.xlabel('Epochs')
plt.ylabel('Average Marker Error')
plt.legend()
plt.show()

plt.plot(metrics['Training Max Marker Error'], label = 'Training Error', marker = '.')
plt.plot(metrics['Validation Max Marker Error'], label = 'Validation Error', marker = '.')
plt.yscale('log')
plt.title('Max Marker Error vs. Epoch')
plt.xlabel('Epochs')
plt.ylabel('Max Marker Error')
plt.legend()
plt.show()